### Import Statements

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
import tensorflow as tf

### Importing Data

### Splitting into training and test data 
The data comes in two different files, one for the training set and one for the testing set. I then seperated it into the X and Y variables. Finally I converted them to numpy arrays in order to use them later to fit the model.

In [2]:
test = pd.read_csv('fashion-mnist_test.csv')
train = pd.read_csv('fashion-mnist_train.csv')
xtest=test.drop(["label"], axis=1)
ytest=test["label"]
xtrain=train.drop(["label"], axis=1)
ytrain=train["label"]
x_train = np.asarray(xtrain)
y_train = np.asarray(ytrain)
x_test = np.asarray(xtest)
y_test = np.asarray(ytest)

# Setting up the Model

### Function setup
This function is to create the desired model and will be called in future with various parameter values. 

In [3]:
def create_model(num_layer, num_neuron):
    model = tf.keras.Sequential()
    #here we are using the truncated normal distribution to set the initial weights
    kernel_initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=2 / np.sqrt(len(x_train) + num_neuron))
    model.add(tf.keras.layers.Dense(num_neuron, input_shape=(28 * 28,), activation='relu', kernel_initializer=kernel_initializer))
    for i in range(num_layer):
        model.add(tf.keras.layers.Dense(num_neuron, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

### Random Search
using random search to find the hyperparameter values with the highest accuracy. I used the entire training set for this because the random search function uses cross validation so it is not necessary to pull a validation set from the training set when choosing the hyperparameters

In [4]:
num_layer = [1, 2, 3, 4]
num_neuron = [96, 112, 128, 144]
# Must create a dictionary that contains the hyperparameters and their values for the param_distributions
param_dist = dict(num_layer=num_layer, num_neuron=num_neuron)
# Creating the estimator to pass into the random search function
model = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, epochs=40, batch_size=500, verbose=0)
random_search = RandomizedSearchCV(estimator=model,param_distributions=param_dist, n_iter=10, cv=3)
random_search.fit(x_train, y_train)

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x1a2fe8c320>,
                   iid='warn', n_iter=10, n_jobs=None,
                   param_distributions={'num_layer': [1, 2, 3, 4],
                                        'num_neuron': [96, 112, 128, 144]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

# Results 

### Random search results
here the results show the highest accuracy was obtained with number of neurons being set to 112 
and number of layers being set to 4

In [5]:
print("Best: %f using %s" % (random_search.best_score_, random_search.best_params_))
means = random_search.cv_results_['mean_test_score']
stds = random_search.cv_results_['std_test_score']
params = random_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.877117 using {'num_neuron': 112, 'num_layer': 4}
0.864883 (0.004818) with: {'num_neuron': 128, 'num_layer': 3}
0.872567 (0.001636) with: {'num_neuron': 144, 'num_layer': 2}
0.865050 (0.003900) with: {'num_neuron': 96, 'num_layer': 2}
0.872417 (0.007068) with: {'num_neuron': 144, 'num_layer': 4}
0.863817 (0.001438) with: {'num_neuron': 96, 'num_layer': 1}
0.877117 (0.002388) with: {'num_neuron': 112, 'num_layer': 4}
0.867267 (0.002243) with: {'num_neuron': 96, 'num_layer': 3}
0.865717 (0.001838) with: {'num_neuron': 112, 'num_layer': 2}
0.865567 (0.005147) with: {'num_neuron': 144, 'num_layer': 1}
0.864033 (0.004308) with: {'num_neuron': 128, 'num_layer': 1}


### Fitting Final Model
now we fit the model with the optimized hyperparameters to the training set 

In [6]:
final_model=create_model(random_search.best_params_["num_layer"], random_search.best_params_["num_neuron"])
final_model.fit(x_train, y_train)

Train on 60000 samples
60000/60000 [==============================] - 7s 117us/sample - loss: 0.6577 - accuracy: 0.7856


### Final Model Results
the results below show that the accuracy on the test set is approximately 81%

In [7]:
results = final_model.evaluate(x_test,y_test, verbose=0)
print("Accuracy:", results[1])

Accuracy: 0.8126
